In [1]:
%load_ext autoreload
%autoreload 2


# Import Libraries

In [2]:
import os

import pandas as pd
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import accuracy_score


# Read Data

In [3]:
path = "../"
train_data = pd.read_csv(os.path.join(path, "train.csv"))
test_data = pd.read_csv(os.path.join(path, "test.csv"))
print(f"Number of rows and columns in the train data set: {train_data.shape}")
print(f"Number of rows and columns in the test data set: {test_data.shape}")
train_data.head()


Number of rows and columns in the train data set: (8050, 4)
Number of rows and columns in the test data set: (2684, 3)


,movie_name,target,movie_description,id
0,Furies,0,Three furious vigilantes unite to take down a ...,133529636342330622371894152500993949030
1,RRR,0,The story of freedom fighters Komaram Bheem an...,133529660110779376651195430564179049830
2,John Wick,0,Legendary assassin John Wick (Keanu Reeves) re...,133529680710101630359923204885606137190
3,John Wick: Chapter 3 -- Parabellum,0,After gunning down a member of the High Table ...,133529687048354631501070212369122164070
4,Top Gun: Maverick,0,After more than thirty years of service as one...,133529699724860633783364227336154217830


In [4]:
train_data.groupby("target").describe()


movie_name                                   movie_description         \
            count unique                   top freq             count unique   
target                                                                         
0             920    908             The Mummy    2               920    920   
1            1096   1075  The Three Musketeers    4              1096   1096   
2            1140   1133             Rembrandt    2              1140   1140   
3            1589   1578        Ocean's Eleven    2              1589   1589   
4            1699   1645             Halloween    3              1699   1699   
5            1606   1590    Always Be My Maybe    2              1606   1606   

                                                                  id         \
                                                      top freq count unique   
target                                                                        
0       Three furious vigilantes unite to take down a ...    1   920    920   
1       An interactive comedy adventure about two indo...    1  1096   1096   
2       Zlatan Ibrahimovic, the son of Balkan immigran...    1  1140   1140   
3       Aware that her time is running short, irascibl...    1  1589   1589   
4       Returning to an eerily empty hotel after a day...    1  1699   1699   
5       In order to save her relationship from falling...    1  1606   1606   

                                                      
                                            top freq  
target                                                
0       133529636342330622371894152500993949030    1  
1       133537091712423214646061704986720566630    1  
2       133546465196330277259842397171484818790    1  
3       133555542366909536525000489501874814310    1  
4       133568186389365187970637043180908936550    1  
5       133581667853498615190321960619498110310    1

# Preparing the data and creating Catboost model

In [5]:
X_train = train_data["movie_description"]
y_train = train_data["target"]

X_test = test_data["movie_description"]


model = CatBoostClassifier(
    iterations=100,
    depth=5,
    random_seed=42
)

model.fit(
    X_train,
    y_train,
    text_features=[0],
    verbose=True
)


Learning rate set to 0.5
0:	learn: 0.6379966	total: 192ms	remaining: 19s
1:	learn: 0.4115786	total: 353ms	remaining: 17.3s
2:	learn: 0.3088364	total: 506ms	remaining: 16.4s
3:	learn: 0.2512520	total: 657ms	remaining: 15.8s
4:	learn: 0.2143676	total: 805ms	remaining: 15.3s
5:	learn: 0.1921620	total: 949ms	remaining: 14.9s
6:	learn: 0.1753263	total: 1.1s	remaining: 14.6s
7:	learn: 0.1564486	total: 1.27s	remaining: 14.7s
8:	learn: 0.1494784	total: 1.43s	remaining: 14.4s
9:	learn: 0.1489393	total: 1.59s	remaining: 14.3s
10:	learn: 0.1453907	total: 1.74s	remaining: 14s
11:	learn: 0.1422216	total: 1.89s	remaining: 13.9s
12:	learn: 0.1372412	total: 2.04s	remaining: 13.7s
13:	learn: 0.1366207	total: 2.19s	remaining: 13.4s
14:	learn: 0.1321891	total: 2.34s	remaining: 13.2s
15:	learn: 0.1309876	total: 2.48s	remaining: 13s
16:	learn: 0.1240254	total: 2.63s	remaining: 12.8s
17:	learn: 0.1222762	total: 2.77s	remaining: 12.6s
18:	learn: 0.1219894	total: 2.92s	remaining: 12.5s
19:	learn: 0.1091740	to

# Predict

In [6]:
# Preparing data in Pool format
dataset_test = Pool(
    data=X_test,
    text_features=[0]
)
predict_classes = model.predict(dataset_test)
predictions = predict_classes


# Create submission

In [7]:
sample_submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))
sample_submission["target"] = predictions
sample_submission.head()


,id,target
0,133529667241314002934985813983134580070,5
1,133529693386607632642217219852638190950,5
2,133529737754378640630246272237250379110,5
3,133529756769137644053687294687798459750,1
4,133529828866765532034234504812793265510,5


In [8]:
sample_submission.to_csv("submission.csv", index=False)
